Multi-layer Perceptron(MLP)

Install Library & requirements

In [1]:
!pwd

/home/jmj3047/mj_MLP_prac


In [2]:
import os
import sys
from datetime import datetime

drive_project_root = "/home/jmj3047/mj_MLP_prac"
sys.path.append(drive_project_root)


In [3]:
!pip install -r "/home/jmj3047/mj_MLP_prac/requirements.txt"

  Using cached wandb-0.11.1-py2.py3-none-any.whl (1.7 MB)
  Using cached torch-1.9.0-cp38-cp38-manylinux1_x86_64.whl (831.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.12
    Uninstalling wandb-0.12.12:
      Successfully uninstalled wandb-0.12.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


In [9]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim

from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import FashionMNIST
from torchvision import transforms

ModuleNotFoundError: No module named 'torch.ao.quantization'

In [7]:
#!pwd
data_root = os.path.join(os.getcwd(), 'data')

#preprocessing & 데이터 셋 정의
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5],[0.5]) #mean, std
    ]
)

fashion_mnist_dataset = FashionMNIST(data_root, download=True, train=True, transform=transform)


NameError: name 'transforms' is not defined